In [84]:
import os
import sys
import re
sys.path.append('/Users/laurenthericourt/projets/trading/trading')

import psycopg2
import pandas as pd
from pandas.io.sql import read_sql
pd.set_option('display.max_rows', 500)
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from stockstats import StockDataFrame

from config.load import load_conf
from db.utils import get_uri_db
from data.candle import SYMBOLS

In [7]:
load_conf('../config/configuration.yaml')

In [8]:
dsn = get_uri_db()
schema = 'trading'

# Get data
## Candles

In [9]:
#SYMBOL = 'EUR/USD'
start_date = '2020-11-01'
end_date = '2020-12-01'

In [10]:
candles = pd.DataFrame()
with psycopg2.connect(dsn) as conn:
    for table in ['candle', 'candle15m', 'candle30m', 'candle1h', 'candle4h', 'candle1d']:
        sql = f'set search_path = {schema};'
        sql += f'''
            SELECT '{table}' as table, symbol, date, open, close, low, high
            FROM {table}
            WHERE date >= %(start_date)s
              AND date < %(end_date)s
            ORDER BY date ASC;
        '''
        candles_tmp = read_sql(sql, conn, params={'start_date': start_date, 'end_date': end_date})
        candles = pd.concat([candles, candles_tmp])

## Event

In [7]:
COUNTRY_1, COUNTRY_2 = SYMBOL.split('/')

In [9]:
with psycopg2.connect(dsn) as conn:
    sql = f'set search_path = {schema};'
    sql += '''
        SELECT date, country, name, actual_value, forecast_value, is_positive
        FROM event
        WHERE importance = 3
        AND country = ANY(%(countries)s::text[])
    '''
    event = read_sql(sql, conn, params={'countries': [COUNTRY_1, COUNTRY_2]})

In [11]:
months = ['(jan.)', '(févr.)', '(mar)', '(avr)', '(mai)', '(juin)', '(juill.)', '(août)', '(sept.)', '(oct.)', '(nov.)', '(déc)']
regex_month = re.compile(r'|'.join(months).replace('(', '\(').replace(')', '\)').replace('.', '\.'))

periods = [' (t1)', ' (t2)', ' (t3)', ' (t4)', ' m1', ' m2', ' m3', ' m4', ' m5', ' m6', ' m7', ' m8', ' m9', ' m10', ' m11', ' m12']
regex_period = re.compile(r'|'.join(periods).replace('(', '\(').replace(')', '\)'))

In [12]:
event['processed_name'] = event['name'].str.lower()
event['processed_name'] = event['processed_name'].str.replace(regex_month, '')
event['processed_name'] = event['processed_name'].str.replace(regex_period, '')
event['processed_name'] = event['processed_name'].str.strip()

In [13]:
event['processed_name'].value_counts()[:10]

stocks de pétrole brut                              571
inscriptions hebdomadaires au chômage               571
ipc (annuel)                                        263
discours de draghi, président de la bce             209
promesses de ventes de logements (mensuel)          132
rapport jolts - nouvelles offres d'emploi           132
indice pmi non manufacturier de l'ism               132
créations d'emplois dans le secteur non agricole    132
indice pmi manufacturier de l'ism                   132
taux de chômage                                     132
Name: processed_name, dtype: int64

# Compute Indicators

# Show data

In [101]:
candles

,table,symbol,date,open,close,low,high
0,candle,EUR/USD,2020-11-01 19:30:00,1.16444,1.16436,1.16444,1.16436
1,candle,USD/CAD,2020-11-01 19:30:00,1.33146,1.33144,1.33166,1.33144
2,candle,AUD/USD,2020-11-01 19:30:00,0.70258,0.70222,0.70254,0.70221
3,candle,USD/JPY,2020-11-01 19:30:00,104.64000,104.51100,104.51100,104.51000
4,candle,GBP/USD,2020-11-01 19:30:00,1.29451,1.29209,1.29225,1.29207
...,...,...,...,...,...,...,...
203,candle1d,USD/CAD,2020-11-30 00:00:00,1.29823,1.29903,1.29304,1.30029
204,candle1d,USD/JPY,2020-11-30 00:00:00,104.08200,104.29900,103.85500,104.39100
205,candle1d,EUR/USD,2020-11-30 00:00:00,1.19715,1.19368,1.19247,1.19952
206,candle1d,EUR/GBP,2020-11-30 00:00:00,0.89788,0.89504,0.89451,0.89950


In [58]:
def show_candle(candles, table, symbol=None, start_date=None, end_date=None):
    mask = (candles['table'] == table)
    if symbol:
        mask = mask & (candles['symbol'] == symbol)
    if start_date and end_date:
        mask = mask & (candles['date'] >= start_date) & (candles['date'] < end_date)
    candles_to_show = candles[mask]
    
    layout = go.Layout(
        autosize=True,
        width=1400,
        height=800,

        xaxis= go.layout.XAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True),

        yaxis= go.layout.YAxis(linecolor = 'black',
                              linewidth = 1,
                              mirror = True),

    )
    
    fig = go.Figure(data=[
                        go.Candlestick(x=candles_to_show['date'],
                        open=candles_to_show['open'],
                        high=candles_to_show['high'],
                        low=candles_to_show['low'],
                        close=candles_to_show['close'])],
                    layout = layout
                   )
    fig.update_yaxes(fixedrange=False)

    #fig.show()
    return fig
    

In [83]:
# Build App
app = JupyterDash()

app.layout = html.Div([
    html.H1(f'Candles between {start_date} and {end_date}'),
    html.Div([
        html.Div([
            html.Label('Symbol:', className='label'),
            dcc.RadioItems(
                id='symbols_radio',
                options=[{'label': x, 'value': x} for x in SYMBOLS],
                value='EUR/USD'
            )], className='col'),
        html.Div([
            html.Label('Period:', className='label'),
            dcc.RadioItems(
                id='period_radio',
                labelStyle={'display': 'inline-block'},
                options=[{'label':'5min', 'value':'candle'},
                         {'label':'15min', 'value':'candle15m'},
                         {'label':'30min', 'value':'candle30m'},
                         {'label':'1h', 'value':'candle1h'},
                         {'label':'4h', 'value':'candle4h'},
                         {'label':'1d', 'value':'candle1d'}],
                value='candle1d'
            )], className='col')
    ], className='row'),
    dcc.Graph(id='candles_graph'),
])

@app.callback(
    Output('candles_graph', 'figure'),
    Input('symbols_radio', 'value'),
    Input('period_radio', 'value'))
def update_figure(selected_symbol, selected_period):
    return show_candle(candles, selected_period, selected_symbol)

app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/


In [89]:
stock = StockDataFrame.retype(candles[candles['table'] == 'candle'])

In [92]:
stock.get('boll')

date
2020-11-01 19:30:00     1.164360
2020-11-01 19:30:00     1.247900
2020-11-01 19:30:00     1.066007
2020-11-01 19:30:00    26.927255
2020-11-01 19:30:00    21.800222
                         ...    
2020-11-30 23:55:00    17.217801
2020-11-30 23:55:00    17.545663
2020-11-30 23:55:00    12.377622
2020-11-30 23:55:00    12.355681
2020-11-30 23:55:00    12.383838
Name: boll, Length: 49315, dtype: float64